<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Visualization**


Estimated time needed: **45** minutes


In this lab, you will focus on data visualization. The dataset will be provided through an RDBMS, and you will need to use SQL queries to extract the required data.


## Objectives


After completing this lab, you will be able to:


-   Visualize the distribution of data.

-   Visualize the relationship between two features.

-   Visualize composition and comparison of data.




## Demo: How to work with database


Download the database file.


In [14]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv

--2025-10-05 20:13:24--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv
169.63.118.104ourses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
200 OKequest sent, awaiting response... 
Length: 159525875 (152M) [text/csv]
Saving to: ‘survey-data.csv.1’

survey-data.csv.1   100%[===================>] 152.13M  59.7MB/s    in 2.6s    

2025-10-05 20:13:29 (59.7 MB/s) - ‘survey-data.csv.1’ saved [159525875/159525875]



**Install and Import Necessary Python Libraries**

Ensure that you have the required libraries installed to work with SQLite and Pandas:


In [15]:
!pip install pandas 
!pip install matplotlib

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

**Read the CSV File into a Pandas DataFrame**

Load the Stack Overflow survey data into a Pandas DataFrame:


In [ ]:
# Read the CSV file
df = pd.read_csv('survey-data.csv')

# Display the first few rows of the data
df.head()


**Create a SQLite Database and Insert the Data**

Now, let's create a new SQLite database (`survey-data.sqlite`) and insert the data from the DataFrame into a table using the sqlite3 library:


In [ ]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect('survey-data.sqlite')

# Write the dataframe to the SQLite database
df.to_sql('main', conn, if_exists='replace', index=False)


# Close the connection
conn.close()


**Verify the Data in the SQLite Database**
Verify that the data has been correctly inserted into the SQLite database by running a simple query:


In [ ]:
# Reconnect to the SQLite database
conn = sqlite3.connect('survey-data.sqlite')

# Run a simple query to check the data
QUERY = "SELECT * FROM main LIMIT 5"
df_check = pd.read_sql_query(QUERY, conn)

# Display the results
print(df_check)


## Demo: Running an SQL Query


Count the number of rows in the table named 'main'


In [ ]:
QUERY = """
SELECT COUNT(*) 
FROM main
"""
df = pd.read_sql_query(QUERY, conn)
df.head()


## Demo: Listing All Tables


To view the names of all tables in the database:


In [ ]:
QUERY = """
SELECT name as Table_Name FROM sqlite_master 
WHERE type = 'table'
"""
pd.read_sql_query(QUERY, conn)


## Demo: Running a Group By Query
    
For example, you can group data by a specific column, like Age, to get the count of respondents in each age group:


In [ ]:
QUERY = """
SELECT Age, COUNT(*) as count
FROM main
GROUP BY Age
ORDER BY Age
"""
pd.read_sql_query(QUERY, conn)


## Demo: Describing a table

Use this query to get the schema of a specific table, main in this case:


In [ ]:
table_name = 'main'

QUERY = """
SELECT sql FROM sqlite_master 
WHERE name= '{}'
""".format(table_name)

df = pd.read_sql_query(QUERY, conn)
print(df.iat[0,0])


## Hands-on Lab


### Visualizing the Distribution of Data

**Histograms**

Plot a histogram of CompTotal (Total Compensation).


In [ ]:
sns.set_theme(style="whitegrid")

# Age → numeric midpoint
age_map = {
    "Under 18 years old": 17,
    "18-24 years old": 21,
    "25-34 years old": 29.5,
    "35-44 years old": 39.5,
    "45-54 years old": 49.5,
    "55-64 years old": 59.5,
    "65 years or older": 70,
}
if "Age_num" not in df.columns and "Age" in df.columns:
    df["Age_num"] = df["Age"].map(age_map)

# Work experience numeric (uses YearsCodePro if WorkExp not present)
def years_to_num(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    if s.startswith("less than"): return 0.5
    if s.startswith("more than"):
        import re
        m = re.search(r"\d+", s);  return float(m.group()) if m else np.nan
    try: return float(s)
    except: return np.nan

if "WorkExp" not in df.columns:
    if "YearsCodePro_num" not in df.columns and "YearsCodePro" in df.columns:
        df["YearsCodePro_num"] = df["YearsCodePro"].apply(years_to_num)
    df["WorkExp"] = df.get("WorkExp", df.get("YearsCodePro_num"))

# Make numeric just in case
for c in ["CompTotal", "TimeSearching", "TimeAnswering", "Frustration", "WorkExp", "Age_num"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

In [ ]:
plt.figure(figsize=(9,5))
sns.histplot(df["CompTotal"].dropna(), bins=40, kde=True)
plt.title("Total Compensation (CompTotal) — Histogram")
plt.xlabel("CompTotal"); plt.ylabel("Count")
plt.tight_layout(); plt.show()## Write your code here

**Box Plots**

Plot a box plot of Age.


In [ ]:
col = "Age_num" if "Age_num" in df.columns else "Age"
plt.figure(figsize=(8,3))
sns.boxplot(x=df[col], orient="h")
plt.title("Age — Box Plot"); plt.xlabel(col)
plt.tight_layout(); plt.show()

### Visualizing Relationships in Data

**Scatter Plots**

Create a scatter plot of Age and WorkExp.


In [ ]:
plot = df.dropna(subset=["Age_num","WorkExp"])
plt.figure(figsize=(7,5))
sns.scatterplot(data=plot, x="Age_num", y="WorkExp", alpha=0.4)
sns.regplot(data=plot, x="Age_num", y="WorkExp", scatter=False, color="red")
plt.title("Age vs Work Experience"); plt.xlabel("Age"); plt.ylabel("WorkExp (years)")
plt.tight_layout(); plt.show()

**Bubble Plots**

Create a bubble plot of `TimeSearching` and `Frustration` using the Age column as the bubble size.


In [ ]:
plot = df.dropna(subset=["TimeSearching","Frustration","Age_num"])
plt.figure(figsize=(7,5))
sns.scatterplot(
    data=plot, x="TimeSearching", y="Frustration",
    size="Age_num", sizes=(20, 400), alpha=0.35, legend=False
)
plt.title("Bubble: TimeSearching vs Frustration (size = Age)")
plt.xlabel("TimeSearching"); plt.ylabel("Frustration")
plt.tight_layout(); plt.show()

### Visualizing Composition of Data

**Pie Charts**

Create a pie chart of the top 5 databases(`DatabaseWantToWorkWith`) that respondents wish to learn next year.


In [ ]:
sep = r"[;|,]"  # typical multi-select separator
databases = (df["DatabaseWantToWorkWith"]
             .dropna()
             .str.split(sep)
             .explode()
             .str.strip()
             .replace("", np.nan)
             .dropna())

top5 = databases.value_counts().head(5)
plt.figure(figsize=(6,6))
top5.plot(kind="pie", autopct="%1.1f%%")
plt.title("Top 5 Databases (want to work with)")
plt.ylabel("")
plt.tight_layout(); plt.show()


**Stacked Charts** 

Create a stacked bar chart of median `TimeSearching` and `TimeAnswering` for the age group 30 to 35.


In [ ]:
age_low, age_high = 30, 35
mask = df["Age_num"].between(age_low, age_high, inclusive="both")

med_ts  = df.loc[mask, "TimeSearching"].median()
med_ta  = df.loc[mask, "TimeAnswering"].median()

stack_df = pd.DataFrame({
    "TimeSearching": [med_ts],
    "TimeAnswering": [med_ta],
}, index=[f"{age_low}-{age_high}"])

ax = stack_df.plot(kind="bar", stacked=True, figsize=(6,4))
plt.title(f"Median Times (Age {age_low}-{age_high})")
plt.xlabel("Age group"); plt.ylabel("Minutes")
plt.legend(title="")
plt.tight_layout(); plt.show()

### Visualizing Comparison of Data

**Line Chart**

Plot the median `CompTotal` for all ages from 45 to 60.


In [ ]:
age_col = "Age_num" if "Age_num" in df.columns else "Age"
age_num = pd.to_numeric(df[age_col], errors="coerce")

sub = df.assign(Age_year=age_num.round(0)).dropna(subset=["CompTotal","Age_year"])
sub = sub[sub["Age_year"].between(45, 60, inclusive="both")]
med_by_age = (sub.groupby("Age_year")["CompTotal"]
                .median().reset_index().sort_values("Age_year"))

plt.figure(figsize=(8,4))
sns.lineplot(data=med_by_age, x="Age_year", y="CompTotal", marker="o")
plt.title("Median CompTotal by Age (45–60)")
plt.xlabel("Age"); plt.ylabel("Median CompTotal")
plt.tight_layout(); plt.show()

**Bar Chart**

Create a horizontal bar chart using the `MainBranch` column.


In [ ]:
counts = df["MainBranch"].value_counts().sort_values()
plt.figure(figsize=(9,6))
sns.barplot(x=counts.values, y=counts.index, orient="h")
plt.title("MainBranch (Horizontal Bar)")
plt.xlabel("Count"); plt.ylabel("")
plt.tight_layout(); plt.show()

### Summary


In this lab, you focused on extracting and visualizing data from an RDBMS using SQL queries and SQLite. You applied various visualization techniques, including:

- Histograms to display the distribution of CompTotal.
- Box plots to show the spread of ages.
- Scatter plots and bubble plots to explore relationships between variables like Age, WorkExp, `TimeSearching` and `TimeAnswering`.
- Pie charts and stacked charts to visualize the composition of data.
- Line charts and bar charts to compare data across categories.


### Close the Database Connection

Once the lab is complete, ensure to close the database connection:


In [ ]:
conn.close()

## Authors:
Ayushi Jain


### Other Contributors:
- Rav Ahuja
- Lakshmi Holla
- Malika


Copyright © IBM Corporation. All rights reserved.
